In [4]:
"""
========================================================================
GRAPH-BASED DEPENDENCY PARSING PROJECT
Maximum Spanning Tree (MST) Algorithm Implementation
========================================================================

Project Overview:
This project implements a classical graph-based dependency parser using
the Chu-Liu-Edmonds algorithm for finding Maximum Spanning Trees. It
demonstrates how dependency parsing was approached before neural methods.

Key Components:
- Chu-Liu-Edmonds MST algorithm
- Structured Perceptron training
- Feature-based scoring system
- Comprehensive evaluation and visualization

Dataset: Universal Dependencies English Treebank (UD_English-EWT)
Install: pip install conllu numpy scipy scikit-learn matplotlib
"""

import numpy as np
from collections import defaultdict, Counter
import re
from typing import List, Tuple, Dict
import pickle
import time
from datetime import datetime

# For Kaggle: Uncomment to download UD English Treebank
# !wget https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-train.conllu
# !wget https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-dev.conllu
# !wget https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-test.conllu

class DependencyTree:
    """Represents a dependency tree structure"""
    def __init__(self, tokens, heads, labels):
        self.tokens = tokens
        self.heads = heads
        self.labels = labels
        
    def __repr__(self):
        result = []
        for i, (token, head, label) in enumerate(zip(self.tokens, self.heads, self.labels)):
            result.append(f"{i}: {token} <- {head} ({label})")
        return "\n".join(result)

class CoNLLReader:
    """Reads CoNLL-U format dependency treebanks"""
    
    @staticmethod
    def read_conllu(filepath):
        sentences = []
        current_tokens = []
        current_heads = []
        current_labels = []
        
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                
                if line.startswith('#') or not line:
                    if current_tokens:
                        sentences.append(DependencyTree(
                            current_tokens, current_heads, current_labels
                        ))
                        current_tokens = []
                        current_heads = []
                        current_labels = []
                    continue
                
                parts = line.split('\t')
                if len(parts) >= 8 and '-' not in parts[0] and '.' not in parts[0]:
                    token = parts[1]
                    head = int(parts[6])
                    label = parts[7]
                    
                    current_tokens.append(token)
                    current_heads.append(head)
                    current_labels.append(label)
        
        if current_tokens:
            sentences.append(DependencyTree(
                current_tokens, current_heads, current_labels
            ))
        
        return sentences

class FeatureExtractor:
    """Extracts features for scoring dependency arcs"""
    
    def __init__(self):
        self.feature_weights = defaultdict(float)
        self.feature_counts = Counter()
        
    def extract_features(self, tokens, head_idx, dep_idx):
        """Extract features for a potential dependency arc"""
        features = []
        
        # Handle ROOT and ensure indices are valid
        if head_idx == 0:
            head_token = "ROOT"
        elif head_idx > 0 and head_idx <= len(tokens):
            head_token = tokens[head_idx - 1]
        else:
            head_token = "UNK"
        
        if dep_idx >= 0 and dep_idx < len(tokens):
            dep_token = tokens[dep_idx]
        else:
            dep_token = "UNK"
        
        # Distance features
        distance = abs(head_idx - dep_idx)
        features.append(f"dist={distance}")
        features.append(f"dist_bin={min(distance, 5)}")
        
        # Direction feature
        direction = "right" if dep_idx > head_idx else "left"
        features.append(f"dir={direction}")
        
        # Lexical features
        features.append(f"head={head_token.lower()}")
        features.append(f"dep={dep_token.lower()}")
        features.append(f"head_dep={head_token.lower()}_{dep_token.lower()}")
        
        # Suffix features
        if len(dep_token) >= 3:
            features.append(f"dep_suffix={dep_token[-3:].lower()}")
        if len(head_token) >= 3 and head_idx > 0:
            features.append(f"head_suffix={head_token[-3:].lower()}")
        
        # Capitalization features
        features.append(f"dep_cap={dep_token[0].isupper()}")
        
        # Position features
        if dep_idx == 0:
            features.append("dep_is_first")
        if dep_idx == len(tokens) - 1:
            features.append("dep_is_last")
        
        return features
    
    def score_arc(self, tokens, head_idx, dep_idx):
        """Score a potential dependency arc"""
        features = self.extract_features(tokens, head_idx, dep_idx)
        score = sum(self.feature_weights[f] for f in features)
        return score

class ChuLiuEdmonds:
    """Implementation of Chu-Liu-Edmonds algorithm for finding MST"""
    
    @staticmethod
    def find_mst(score_matrix):
        """Find maximum spanning tree using Chu-Liu-Edmonds algorithm"""
        n = len(score_matrix)
        
        best_heads = [0] * n
        for j in range(1, n):
            best_score = float('-inf')
            best_head = 0
            for i in range(n):
                if i != j and score_matrix[i][j] > best_score:
                    best_score = score_matrix[i][j]
                    best_head = i
            best_heads[j] = best_head
        
        visited = [False] * n
        cycle = ChuLiuEdmonds._find_cycle(best_heads, visited)
        
        if cycle is None:
            return best_heads
        
        return ChuLiuEdmonds._contract_cycle(score_matrix, best_heads, cycle)
    
    @staticmethod
    def _find_cycle(heads, visited):
        """Detect cycle in the current tree"""
        n = len(heads)
        for start in range(1, n):
            if visited[start]:
                continue
            
            path = []
            current = start
            
            while current not in path and not visited[current]:
                path.append(current)
                current = heads[current]
                if current == 0:
                    break
            
            if current in path:
                cycle_start = path.index(current)
                return path[cycle_start:]
            
            for node in path:
                visited[node] = True
        
        return None
    
    @staticmethod
    def _contract_cycle(score_matrix, heads, cycle):
        """Contract a cycle and recursively find MST"""
        n = len(score_matrix)
        cycle_set = set(cycle)
        
        non_cycle = [i for i in range(n) if i not in cycle_set]
        new_node_idx = len(non_cycle)
        new_size = len(non_cycle) + 1
        
        new_score_matrix = [[float('-inf')] * new_size for _ in range(new_size)]
        
        old_to_new = {}
        for idx, node in enumerate(non_cycle):
            old_to_new[node] = idx
        for node in cycle:
            old_to_new[node] = new_node_idx
        
        for i in range(n):
            for j in range(n):
                if i == j:
                    continue
                new_i = old_to_new[i]
                new_j = old_to_new[j]
                
                if new_i == new_j:
                    continue
                
                score = score_matrix[i][j]
                
                if j in cycle_set and i not in cycle_set:
                    score -= score_matrix[heads[j]][j]
                
                new_score_matrix[new_i][new_j] = max(
                    new_score_matrix[new_i][new_j], score
                )
        
        new_heads = ChuLiuEdmonds.find_mst(new_score_matrix)
        
        final_heads = list(heads)
        
        for node in cycle:
            if old_to_new[heads[node]] != new_node_idx:
                pass
            elif new_heads[new_node_idx] != new_node_idx:
                external_head = non_cycle[new_heads[new_node_idx]] if new_heads[new_node_idx] < new_node_idx else None
                if external_head is not None:
                    final_heads[node] = external_head
                    break
        
        return final_heads

class MSTParser:
    """Graph-based dependency parser using MST algorithm"""
    
    def __init__(self):
        self.feature_extractor = FeatureExtractor()
        self.training_stats = {
            'epoch_accuracies': [],
            'epoch_times': [],
            'total_updates': 0
        }
        
    def train(self, train_trees, epochs=10, learning_rate=0.1, verbose=True):
        """Train the parser using structured perceptron"""
        print("\n" + "="*80)
        print("TRAINING PHASE")
        print("="*80)
        print(f"Training sentences: {len(train_trees)}")
        print(f"Epochs: {epochs}")
        print(f"Learning rate: {learning_rate}")
        print(f"Algorithm: Structured Perceptron with MST inference")
        print("="*80 + "\n")
        
        for epoch in range(epochs):
            epoch_start = time.time()
            correct_arcs = 0
            total_arcs = 0
            updates = 0
            
            for tree_idx, gold_tree in enumerate(train_trees):
                if tree_idx % 100 == 0 and verbose:
                    print(f"Epoch {epoch+1}/{epochs} | Sentence {tree_idx}/{len(train_trees)} | "
                          f"Updates: {updates}", end='\r')
                
                predicted_heads = self.parse(gold_tree.tokens)
                
                if len(predicted_heads) != len(gold_tree.tokens):
                    continue
                
                n = len(gold_tree.tokens)
                for dep_idx in range(n):
                    gold_head = gold_tree.heads[dep_idx]
                    pred_head = predicted_heads[dep_idx]
                    
                    if pred_head < 0 or pred_head > n:
                        continue
                    
                    if gold_head == pred_head:
                        correct_arcs += 1
                    else:
                        # Update for gold arc
                        gold_features = self.feature_extractor.extract_features(
                            gold_tree.tokens, gold_head, dep_idx
                        )
                        for feat in gold_features:
                            self.feature_extractor.feature_weights[feat] += learning_rate
                            self.feature_extractor.feature_counts[feat] += 1
                        
                        # Update for predicted arc
                        pred_features = self.feature_extractor.extract_features(
                            gold_tree.tokens, pred_head, dep_idx
                        )
                        for feat in pred_features:
                            self.feature_extractor.feature_weights[feat] -= learning_rate
                            self.feature_extractor.feature_counts[feat] += 1
                        
                        updates += 1
                    
                    total_arcs += 1
            
            epoch_time = time.time() - epoch_start
            accuracy = correct_arcs / total_arcs if total_arcs > 0 else 0
            
            self.training_stats['epoch_accuracies'].append(accuracy)
            self.training_stats['epoch_times'].append(epoch_time)
            self.training_stats['total_updates'] += updates
            
            print(f"\nEpoch {epoch+1}/{epochs} Complete:")
            print(f"  ├─ Accuracy: {accuracy:.4f} ({correct_arcs}/{total_arcs})")
            print(f"  ├─ Updates: {updates}")
            print(f"  ├─ Time: {epoch_time:.2f}s")
            print(f"  └─ Sentences/sec: {len(train_trees)/epoch_time:.1f}")
        
        print("\n" + "="*80)
        print("TRAINING COMPLETE")
        print("="*80)
        print(f"Total updates: {self.training_stats['total_updates']}")
        print(f"Active features: {len(self.feature_extractor.feature_weights)}")
        print(f"Final accuracy: {self.training_stats['epoch_accuracies'][-1]:.4f}")
        print("="*80 + "\n")
        
    def parse(self, tokens):
        """Parse a sentence and return head indices"""
        n = len(tokens)
        score_matrix = [[float('-inf')] * (n + 1) for _ in range(n + 1)]
        
        for j in range(1, n + 1):
            score_matrix[0][j] = self.feature_extractor.score_arc(tokens, 0, j - 1)
        
        for i in range(1, n + 1):
            for j in range(1, n + 1):
                if i != j:
                    score_matrix[i][j] = self.feature_extractor.score_arc(tokens, i, j - 1)
        
        heads = ChuLiuEdmonds.find_mst(score_matrix)
        
        token_heads = []
        for i in range(1, n + 1):
            head = heads[i]
            if head < 0 or head > n:
                head = 0
            token_heads.append(head)
        
        return token_heads
    
    def evaluate(self, test_trees, detailed=True):
        """Evaluate parser on test set with detailed metrics"""
        correct = 0
        total = 0
        error_types = Counter()
        distance_errors = []
        
        for tree in test_trees:
            predicted_heads = self.parse(tree.tokens)
            for i, (pred_head, gold_head) in enumerate(zip(predicted_heads, tree.heads)):
                if pred_head == gold_head:
                    correct += 1
                else:
                    # Categorize error
                    gold_dist = abs(gold_head - (i + 1))
                    pred_dist = abs(pred_head - (i + 1))
                    distance_errors.append((gold_dist, pred_dist))
                    
                    if gold_head == 0:
                        error_types['wrong_root'] += 1
                    elif pred_head == 0:
                        error_types['false_root'] += 1
                    else:
                        error_types['wrong_attachment'] += 1
                
                total += 1
        
        uas = correct / total if total > 0 else 0
        
        if detailed:
            print(f"\nUnlabeled Attachment Score (UAS): {uas:.4f}")
            print(f"Correct attachments: {correct}/{total}")
            print(f"\nError Breakdown:")
            print(f"  ├─ Wrong root: {error_types['wrong_root']}")
            print(f"  ├─ False root: {error_types['false_root']}")
            print(f"  └─ Wrong attachment: {error_types['wrong_attachment']}")
        
        return uas, error_types
    
    def analyze_features(self, top_k=20):
        """Analyze learned feature weights"""
        print("\n" + "="*80)
        print("FEATURE ANALYSIS")
        print("="*80)
        
        sorted_features = sorted(self.feature_extractor.feature_weights.items(), 
                                key=lambda x: abs(x[1]), reverse=True)
        
        print(f"\nTop {top_k} Most Important Features:")
        print("-" * 80)
        print(f"{'Feature':<50} {'Weight':>15} {'Count':>10}")
        print("-" * 80)
        
        for feat, weight in sorted_features[:top_k]:
            count = self.feature_extractor.feature_counts[feat]
            print(f"{feat:<50} {weight:>15.4f} {count:>10}")
        
        print("\n" + "="*80 + "\n")
    
    def save(self, filepath):
        """Save model to file"""
        with open(filepath, 'wb') as f:
            pickle.dump({
                'weights': self.feature_extractor.feature_weights,
                'stats': self.training_stats
            }, f)
        print(f"Model saved to {filepath}")
    
    def load(self, filepath):
        """Load model from file"""
        with open(filepath, 'rb') as f:
            data = pickle.load(f)
            self.feature_extractor.feature_weights = data['weights']
            self.training_stats = data['stats']
        print(f"Model loaded from {filepath}")

def visualize_parse_tree(tokens, heads, labels=None):
    """Create ASCII art visualization of dependency tree"""
    n = len(tokens)
    
    print("\n" + "="*80)
    print("DEPENDENCY TREE VISUALIZATION")
    print("="*80)
    
    # Print tokens with indices
    print("\nTokens:")
    for i, token in enumerate(tokens):
        print(f"  [{i+1}] {token}")
    
    print("\nDependency Arcs:")
    print("-" * 80)
    print(f"{'Dependent':<15} {'Head':<15} {'Relation':<15} {'Direction':<10}")
    print("-" * 80)
    
    for i, (token, head) in enumerate(zip(tokens, heads)):
        head_token = "ROOT" if head == 0 else tokens[head - 1]
        label = labels[i] if labels else "dep"
        direction = "←" if head < i + 1 else "→" if head > i + 1 else "↓"
        
        print(f"{token:<15} {head_token:<15} {label:<15} {direction:<10}")
    
    print("="*80 + "\n")

def test_on_diverse_examples(parser):
    """Comprehensive testing on diverse linguistic phenomena"""
    
    test_cases = [
        {
            'category': 'Simple SVO',
            'sentences': [
                ["I", "saw", "her"],
                ["Dogs", "bark", "loudly"],
                ["The", "cat", "sleeps"],
            ]
        },
        {
            'category': 'Prepositional Phrases (PP-Attachment)',
            'sentences': [
                ["I", "saw", "the", "man", "with", "the", "telescope"],
                ["She", "put", "the", "book", "on", "the", "table"],
            ]
        },
        {
            'category': 'Relative Clauses',
            'sentences': [
                ["The", "dog", "that", "I", "saw", "was", "big"],
                ["People", "who", "live", "here", "are", "happy"],
            ]
        },
        {
            'category': 'Coordination',
            'sentences': [
                ["John", "and", "Mary", "went", "home"],
                ["She", "quickly", "and", "quietly", "left"],
            ]
        },
        {
            'category': 'Long-Distance Dependencies',
            'sentences': [
                ["What", "did", "you", "think", "he", "said"],
                ["Who", "do", "you", "believe", "won"],
            ]
        },
        {
            'category': 'Complex Sentences',
            'sentences': [
                ["I", "think", "that", "he", "is", "smart"],
                ["The", "horse", "raced", "past", "the", "barn", "fell"],
            ]
        }
    ]
    
    print("\n" + "="*80)
    print("COMPREHENSIVE TESTING ON DIVERSE LINGUISTIC PHENOMENA")
    print("="*80)
    
    for test_group in test_cases:
        print(f"\n{'─'*80}")
        print(f"Category: {test_group['category']}")
        print(f"{'─'*80}")
        
        for sent_tokens in test_group['sentences']:
            print(f"\nSentence: {' '.join(sent_tokens)}")
            heads = parser.parse(sent_tokens)
            
            print("Parse:")
            for i, (token, head) in enumerate(zip(sent_tokens, heads)):
                head_token = "ROOT" if head == 0 else sent_tokens[head - 1]
                arrow = "←" if head < i + 1 else "→" if head > i + 1 else "↓"
                print(f"  {token:15s} {arrow} {head_token:15s}")
    
    print("\n" + "="*80)
    print("KNOWN FAILURE MODES AND LIMITATIONS")
    print("="*80)
    
    failure_analysis = """
    1. PP-ATTACHMENT AMBIGUITY
       Example: "I saw the man with the telescope"
       Challenge: Should 'telescope' attach to 'saw' (instrument) or 'man' (possession)?
       Why it fails: Requires semantic knowledge about plausibility
       
    2. LONG-DISTANCE DEPENDENCIES
       Example: "What did you think that he said"
       Challenge: 'What' is the object of 'said', several words away
       Why it fails: Non-local dependencies hard to capture with local features
       
    3. COORDINATION AMBIGUITY
       Example: "old men and women"
       Challenge: Does 'old' modify just 'men' or both 'men and women'?
       Why it fails: Requires understanding of scope and semantic parallelism
       
    4. GARDEN PATH SENTENCES
       Example: "The horse raced past the barn fell"
       Challenge: 'raced' is initially parsed as main verb, but is actually reduced relative
       Why it fails: No reanalysis mechanism; commits to first interpretation
       
    5. LIMITED LEXICAL SEMANTICS
       Example: Semantic role assignment, verb subcategorization
       Challenge: Different verbs have different argument structures
       Why it fails: No access to lexical semantic databases or embeddings
       
    IMPROVEMENTS THAT WOULD HELP:
    - Rich POS tags and morphological features
    - Word embeddings or distributional semantics
    - Syntactic category information
    - Lexicalized grammar knowledge
    - Larger context windows
    - Neural scoring functions
    """
    
    print(failure_analysis)
    print("="*80 + "\n")
    
    # Add detailed comparison with modern approaches
    print_where_classical_models_lag()

def generate_project_report(parser, train_trees, dev_trees, test_trees):
    """Generate comprehensive project report"""
    
    print("\n" + "╔" + "="*78 + "╗")
    print("║" + " "*15 + "GRAPH-BASED DEPENDENCY PARSING PROJECT REPORT" + " "*16 + "║")
    print("╚" + "="*78 + "╝")
    
    print(f"\nProject Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Algorithm: Maximum Spanning Tree (Chu-Liu-Edmonds)")
    print(f"Training Method: Structured Perceptron")
    
    print("\n" + "─"*80)
    print("DATASET STATISTICS")
    print("─"*80)
    
    print(f"Training sentences: {len(train_trees)}")
    print(f"Development sentences: {len(dev_trees)}")
    print(f"Test sentences: {len(test_trees)}")
    
    train_tokens = sum(len(tree.tokens) for tree in train_trees)
    print(f"Training tokens: {train_tokens}")
    print(f"Average sentence length: {train_tokens/len(train_trees):.1f} tokens")
    
    # Analyze dependency label distribution
    label_counts = Counter()
    for tree in train_trees:
        label_counts.update(tree.labels)
    
    print(f"\nMost common dependency relations:")
    for label, count in label_counts.most_common(10):
        print(f"  {label:<15} {count:>6} ({count/sum(label_counts.values())*100:.1f}%)")
    
    print("\n" + "─"*80)
    print("MODEL STATISTICS")
    print("─"*80)
    
    print(f"Total features learned: {len(parser.feature_extractor.feature_weights)}")
    print(f"Total weight updates: {parser.training_stats['total_updates']}")
    print(f"Training epochs: {len(parser.training_stats['epoch_accuracies'])}")
    
    # Feature analysis
    parser.analyze_features(top_k=15)
    
    print("\n" + "="*80 + "\n")

def print_where_classical_models_lag():
    """Detailed analysis of where classical graph-based parsers lag behind modern approaches"""
    
    print("\n" + "╔" + "="*78 + "╗")
    print("║" + " "*10 + "WHERE CLASSICAL GRAPH-BASED PARSERS LAG BEHIND" + " "*20 + "║")
    print("╚" + "="*78 + "╝\n")
    
    print("="*80)
    print("1. FEATURE ENGINEERING BOTTLENECK")
    print("="*80)
    print("""
Classical Approach (This Model):
  • Manual feature templates (distance, lexical, suffix features)
  • Sparse, high-dimensional feature vectors
  • Human expert knowledge required
  • Cannot capture complex feature interactions
  • Limited to predefined feature combinations

Modern Neural Approach:
  • Automatic feature learning via embeddings
  • Dense, continuous representations
  • Captures complex non-linear patterns
  • Pre-trained contextualized embeddings (BERT, RoBERTa)
  • Multi-head attention captures arbitrary feature interactions

Performance Gap: 
  • Classical MST: 60-75% UAS with basic features, 85-90% with engineered features
  • Neural (BiLSTM): 93-95% UAS
  • Transformer-based: 96-98% UAS

Real Example Failure:
  "I deposited the check at the bank"
  Classical: Likely attaches "bank" to "deposited" (correct)
  
  "I sat by the bank"  
  Classical: May still attach "bank" to "sat" using same features
  ❌ Cannot distinguish financial vs. river bank without semantic context
    """)
    
    print("\n" + "="*80)
    print("2. INABILITY TO CAPTURE SEMANTIC SIMILARITY")
    print("="*80)
    print("""
Classical Approach:
  • Treats "dog" and "cat" as completely different symbols
  • No notion of semantic relatedness
  • Cannot generalize across similar words
  • Sparse data problem: unseen word pairs

Modern Approach:
  • Word embeddings place similar words nearby in vector space
  • "dog" and "cat" have similar representations
  • Generalizes patterns learned from "dog" to "cat"
  • Pre-training on massive corpora provides semantic knowledge

Example:
  Training: "The dog chased the cat"
  
  Test: "The wolf hunted the rabbit"
  
  Classical: ❌ Treats "wolf" and "hunted" as completely novel
  Neural: ✓ Recognizes similarity to "dog" and "chased"
  
Impact: 
  Classical models suffer 10-15% accuracy drop on out-of-vocabulary words
  Neural models maintain 95%+ accuracy even with 20% OOV rate
    """)
    
    print("\n" + "="*80)
    print("3. LIMITED CONTEXT WINDOW")
    print("="*80)
    print("""
Classical Approach:
  • Features typically look at 2-3 word window
  • Cannot capture long-range dependencies effectively
  • Context beyond immediate neighbors is hard to encode
  • Exponential feature explosion with larger windows

Modern Approach:
  • BiLSTMs: Can access full sentence context
  • Transformers: Self-attention over entire sequence
  • No limit on context size
  • Efficiently models arbitrarily long dependencies

Example Failure:
  "The keys to the cabinet that the man who lives downstairs owns are missing"
  
  Classical: Struggles to connect "keys" with "are" (10 words apart)
  ❌ Features can't span that distance without exponential combinations
  
  Neural: ✓ Attention mechanism directly connects distant words
    """)
    
    print("\n" + "="*80)
    print("4. GREEDY STRUCTURED PREDICTION")
    print("="*80)
    print("""
Classical MST Parser:
  • Finds globally optimal tree given arc scores
  • BUT arc scores are independently computed
  • Cannot model inter-arc dependencies during scoring
  • Structured perceptron updates are post-hoc corrections

Modern Approaches:
  • Biaffine attention: Jointly scores all potential arcs
  • Graph neural networks: Iteratively refine arc scores
  • Structured training objectives that directly optimize tree accuracy
  • End-to-end differentiable training

Problem Example:
  In "John and Mary went home", if model incorrectly scores:
    - "John" → ROOT (high score)
    - "Mary" → ROOT (high score)
  
  Classical: ❌ MST must pick one, but both have high independent scores
  No mechanism to say "if John is root, penalize Mary being root"
  
  Neural: ✓ Can learn that coordinated subjects shouldn't both be roots
    """)
    
    print("\n" + "="*80)
    print("5. NO TRANSFER LEARNING OR PRE-TRAINING")
    print("="*80)
    print("""
Classical Approach:
  • Trained from scratch on labeled dependency data
  • Requires large annotated treebanks (10,000+ sentences)
  • Cannot leverage unlabeled text
  • Separate models for each language

Modern Approach:
  • Pre-train on billions of tokens of unlabeled text
  • Fine-tune on small labeled datasets (1,000 sentences can suffice)
  • Transfer learning across languages (multilingual BERT)
  • Can leverage knowledge from related tasks

Impact on Low-Resource Languages:
  Classical: Needs 10K+ sentences → 75% UAS
  Neural (from scratch): Same requirement → 85% UAS
  Neural (pre-trained): Just 1K sentences → 90% UAS
  Neural (multilingual): Even works zero-shot! → 75-80% UAS
    """)
    
    print("\n" + "="*80)
    print("6. FIXED FEATURE REPRESENTATIONS")
    print("="*80)
    print("""
Classical Approach:
  • Word "bank" always has same features
  • Cannot handle polysemy (words with multiple meanings)
  • Context-independent representation
  • Ambiguity must be resolved by parser, not representation

Modern Contextualized Embeddings:
  • "bank" has different representations in different contexts
  • BERT/ELMo create context-dependent embeddings
  • Ambiguity partially resolved before parsing
  • Richer input to parser

Example:
  "I went to the bank to deposit money"
  "I sat by the river bank"
  
  Classical: "bank" → same features in both
  ❌ Parser must figure out the difference from limited context features
  
  BERT: "bank" → different embeddings (financial institution vs. riverbank)
  ✓ Pre-resolved ambiguity helps parser make correct attachment
    """)
    
    print("\n" + "="*80)
    print("7. TRAINING DATA EFFICIENCY")
    print("="*80)
    print("""
Classical Perceptron:
  • Requires many passes over data (5-20 epochs)
  • Slow convergence on complex patterns
  • Prone to overfitting on rare features
  • Hand-crafted features may not capture patterns in data

Modern Neural Networks:
  • More sample efficient with pre-training
  • Gradient-based optimization converges faster
  • Regularization techniques (dropout, batch norm) prevent overfitting
  • Learns optimal features from data

Training Time Comparison (to 90% UAS):
  Classical MST: 10K sentences × 15 epochs = 150K sentence-epochs
  Neural (scratch): 10K sentences × 30 epochs = 300K sentence-epochs
  Neural (pre-trained): 2K sentences × 10 epochs = 20K sentence-epochs
    """)
    
    print("\n" + "="*80)
    print("8. HANDLING OF RARE PHENOMENA")
    print("="*80)
    print("""
Classical Sparse Features:
  • Rare word pairs get few training examples
  • Model cannot generalize from similar contexts
  • Effectively backs off to simple heuristics
  • Example: Rare verbs parsed using generic "verb" patterns

Neural Dense Representations:
  • Rare words still have meaningful embeddings
  • Similar words provide indirect evidence
  • Continuous space allows interpolation
  • Compositional understanding helps with novel constructions

Example:
  Rare verb "scrutinize" appears once in training
  
  Classical: ❌ Creates features like "scrutinize_OBJ" with single training example
  Cannot generalize
  
  Neural: ✓ "scrutinize" embedding is close to "examine", "inspect"
  Learns from all similar verbs → better generalization
    """)
    
    print("\n" + "="*80)
    print("9. CROSS-LINGUAL PARSING")
    print("="*80)
    print("""
Classical Approach:
  • Completely separate models per language
  • Features like word forms are language-specific
  • No knowledge transfer between languages
  • Need large treebank for each new language

Modern Multilingual Models:
  • Single model handles 100+ languages
  • Shared multilingual embeddings (mBERT, XLM-R)
  • Zero-shot transfer: Train on one language, parse another
  • Low-resource languages benefit from high-resource ones

Zero-Shot Performance:
  Train on English, test on Spanish:
    Classical: 0% (no Spanish vocabulary)
    Multilingual BERT: 75-80% UAS
    
  This is revolutionary for low-resource languages!
    """)
    
    print("\n" + "="*80)
    print("10. PRACTICAL DEPLOYMENT CONSIDERATIONS")
    print("="*80)
    print("""
Classical Model Advantages:
  ✓ Fast inference (MST algorithm is O(n²))
  ✓ Small model size (few MB)
  ✓ Interpretable features
  ✓ No GPU required
  ✓ Deterministic output

Classical Model Disadvantages:
  ❌ 10-15% lower accuracy
  ❌ Requires linguistic expertise for feature engineering
  ❌ Hard to improve beyond certain accuracy ceiling
  ❌ Separate preprocessing pipeline (POS tagging, etc.)

Modern Neural Model Tradeoffs:
  ✓ State-of-the-art accuracy
  ✓ End-to-end training
  ✓ Easily adaptable to new domains
  
  ❌ Slower inference (especially transformers)
  ❌ Large model size (100MB - 1GB)
  ❌ Requires GPU for practical speed
  ❌ Black box (hard to interpret)

Use Cases:
  Classical: Educational, resource-constrained, need interpretability
  Neural: Production systems where accuracy is critical
    """)
    
    print("\n" + "="*80)
    print("SUMMARY: PERFORMANCE COMPARISON")
    print("="*80)
    print("""
Model Type                           UAS (English)    Speed       Model Size
─────────────────────────────────────────────────────────────────────────────
Classical MST (basic features)          65-75%        1000 sent/s     5 MB
Classical MST (engineered features)     85-90%        500 sent/s      50 MB
Neural BiLSTM (no pre-train)            91-93%        100 sent/s      100 MB
Neural BiLSTM + BiAffine                93-95%        80 sent/s       150 MB
BERT-based (pre-trained)                96-98%        20 sent/s       400 MB
Current SOTA (2024)                     98-99%        10 sent/s       1 GB

Key Insight:
The gap from 90% → 98% accuracy required fundamentally different approaches:
  • Continuous representations instead of sparse features
  • Neural composition instead of manual feature templates  
  • Pre-training instead of training from scratch
  • End-to-end learning instead of pipeline systems
    """)
    
    print("\n" + "╔" + "="*78 + "╗")
    print("║" + " "*15 + "CONCLUSION: WHY NEURAL PARSERS DOMINATE" + " "*23 + "║")
    print("╚" + "="*78 + "╝\n")
    
    print("""
Classical graph-based parsers were a major advance in the 2000s, but they
fundamentally cannot compete with modern neural approaches because:

1. They cannot learn semantic representations from data
2. They require manual feature engineering by experts
3. They cannot efficiently use unlabeled data
4. They cannot transfer knowledge across languages or domains
5. They plateau at ~90% accuracy regardless of data size

Neural parsers overcome ALL of these limitations, achieving near-human
performance (98-99% UAS) on standard benchmarks.

However, classical parsers remain valuable for:
  • Understanding the foundations of dependency parsing
  • Educational purposes
  • Resource-constrained environments
  • Interpretable systems where you need to explain decisions
  • Baseline comparisons in research

This project demonstrates both the elegance of classical algorithms and
their inherent limitations that motivated the neural revolution.
    """)

# Main execution
if __name__ == "__main__":
    print("\n" + "╔" + "="*78 + "╗")
    print("║" + " "*10 + "GRAPH-BASED DEPENDENCY PARSER - MST ALGORITHM PROJECT" + " "*13 + "║")
    print("╚" + "="*78 + "╝\n")
    
    train_file = "en_ewt-ud-train.conllu"
    dev_file = "en_ewt-ud-dev.conllu"
    test_file = "en_ewt-ud-test.conllu"
    
    try:
        # Load data
        print("PHASE 1: DATA LOADING")
        print("─"*80)
        train_trees = CoNLLReader.read_conllu(train_file)
        print(f"✓ Loaded {len(train_trees)} training sentences")
        
        dev_trees = CoNLLReader.read_conllu(dev_file)
        print(f"✓ Loaded {len(dev_trees)} development sentences")
        
        test_trees = CoNLLReader.read_conllu(test_file)
        print(f"✓ Loaded {len(test_trees)} test sentences")
        
        # Initialize parser
        parser = MSTParser()
        
        # Train (use subset for demo, remove [:1000] for full training)
        parser.train(train_trees[:1000], epochs=5, learning_rate=0.1)
        
        # Evaluate
        print("\nPHASE 2: EVALUATION")
        print("─"*80)
        print("\nDevelopment Set Evaluation:")
        dev_uas, dev_errors = parser.evaluate(dev_trees[:200], detailed=True)
        
        print("\nTest Set Evaluation:")
        test_uas, test_errors = parser.evaluate(test_trees[:200], detailed=True)
        
        # Save model
        parser.save("mst_parser_model.pkl")
        
        # Detailed testing
        print("\nPHASE 3: QUALITATIVE ANALYSIS")
        print("─"*80)
        test_on_diverse_examples(parser)
        
        # Example visualization
        print("\nPHASE 4: EXAMPLE PARSE VISUALIZATION")
        example_sent = ["The", "dog", "chased", "the", "cat", "in", "the", "garden"]
        example_heads = parser.parse(example_sent)
        visualize_parse_tree(example_sent, example_heads)
        
        # Generate report
        generate_project_report(parser, train_trees[:1000], dev_trees[:200], test_trees[:200])
        
        print("\n" + "╔" + "="*78 + "╗")
        print("║" + " "*25 + "PROJECT COMPLETE" + " "*36 + "║")
        print("╚" + "="*78 + "╝\n")
        
    except FileNotFoundError:
        print("\n❌ Error: Could not find data files.")
        print("\nPlease run these commands first:")
        print("─"*80)
        print("!wget https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-train.conllu")
        print("!wget https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-dev.conllu")
        print("!wget https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-test.conllu")
        print("─"*80)


╔==============================================================================╗
║          GRAPH-BASED DEPENDENCY PARSER - MST ALGORITHM PROJECT             ║
╚==============================================================================╝

PHASE 1: DATA LOADING
────────────────────────────────────────────────────────────────────────────────
✓ Loaded 12544 training sentences
✓ Loaded 2001 development sentences
✓ Loaded 2077 test sentences

TRAINING PHASE
Training sentences: 1000
Epochs: 5
Learning rate: 0.1
Algorithm: Structured Perceptron with MST inference

Epoch 1/5 | Sentence 900/1000 | Updates: 14196
Epoch 1/5 Complete:
  ├─ Accuracy: 0.2971 (6493/21857)
  ├─ Updates: 15364
  ├─ Time: 3.72s
  └─ Sentences/sec: 269.1
Epoch 2/5 | Sentence 900/1000 | Updates: 12306
Epoch 2/5 Complete:
  ├─ Accuracy: 0.3887 (8496/21857)
  ├─ Updates: 13361
  ├─ Time: 3.70s
  └─ Sentences/sec: 270.1
Epoch 3/5 | Sentence 900/1000 | Updates: 11334
Epoch 3/5 Complete:
  ├─ Accuracy: 0.4387 (9589/21857)
